# TextRank 활용 키워드 추출

In [1]:
# 우선 TFIDF를 활용해 해보자
from konlpy.tag import Komoran


## Mecab 설치 # 나는 class라는 폴더 안에 설치했다.,.

In [2]:
!pip install mecab_python-0.996_ko_0.9.2_msvc-cp36-cp36m-win_amd64.whl

In [3]:
import MeCab

In [4]:
n = MeCab.Tagger()
out = n.parse("메캎이 잘 실행되는지 확인 중입니 다다다다 다")
print(out)

메	VV+EC,*,F,메,Inflect,VV,EC,메/VV/*+어/EC/*
캎이	UNKNOWN,*,*,*,*,*,*,*
잘	MAG,*,T,잘,*,*,*,*
실행	NNG,행위,T,실행,*,*,*,*
되	XSV,*,F,되,*,*,*,*
는지	EC,*,F,는지,*,*,*,*
확인	NNG,행위,T,확인,*,*,*,*
중	NNB,*,T,중,*,*,*,*
입	VV,*,T,입,*,*,*,*
니	EC,*,F,니,*,*,*,*
다다다다	MAG,*,F,다다다다,*,*,*,*
다	MAG,성분부사|정도부사,F,다,*,*,*,*
EOS



## 실습 1, TF-IDF 활용 핵심키워드추출

In [19]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    text = soup.select('div._article_body_contents')[0].text
    return text

docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

5

In [12]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108'

resp = requests.get(url)

soup = BeautifulSoup(resp.content, 'lxml')

text = soup.select('div._article_body_contents')

In [20]:
docs

['\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다. 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다. 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다. 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 

In [30]:
from konlpy.tag import Mecab
mecab = Mecab()

Exception: The MeCab dictionary does not exist at "/usr/local/lib/mecab/dic/mecab-ko-dic". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"

In [ ]:
preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  pass
preprocessed_docs[0][:100]